In [ ]:
import bittensor
import random
import torch
import time

In [ ]:
# Before this step run a pool chain
# ./target/release/node-subtensor --dev

# Make a subtensor connection to the pool chain.
# This is local because we are assuming you ran the chain locally.
# In practice you want to run about 10 validators for your pool on pretty beefy machines.
poolchain = bittensor.subtensor( network = 'local')
poolchain.connect()

# Load all the nodes from the pool. Members of this pool will register themselves
# onto the pool chain and be discovered by by this endpoint.
# bellow we sync with the pool chain.
poolgraph = bittensor.metagraph().sync( subtensor = poolchain )

In [ ]:
# Next create a connection to nobunaga, for testing
# Fisrt create the wallet key using the cli
# btcli new_coldkey --wallet.name poolcoldkey
# btcli new_hotkey --wallet.name poolhotkey
nobunaga = bittensor.subtensor( network = 'nobunaga' )
poolwallet = bittensor.wallet( name = 'poolwallet', hotkey = 'poolhotkey' )

# Register the pool wallet to nobunaga
poolwallet.register( subtensor = nobunaga )

# Register the pool wallet on the poolchain
# This does not link these accounts, but it would be great if we could create a bridge programatically
# something like wallet.bridge( nobunaga, poolchain )
poolwallet.register( subtensor = poolchain )

In [ ]:
# We need an RPC client to forward requests onward.
dendrite = bittensor.dendrite( wallet = poolwallet )

# We are updating weights for each miner, this is nothing crazy, 
# we are just going to use a code base scoring.
moving_average_delta = 0.9
poolscores = torch.zeros_like( poolgraph.stake )

In [ ]:
# Forward is the function that recieves calls from peers on nobunaga
def forward_text ( inputs_x ):
    # Randomly select a peer from the poolgraph.
    # Note that this not weighted, it is purely random.
    endpoint = random.choice( poolgraph.endpoints )
    
    # Query the pool
    response, time, code = dendrite.forward_text( inputs = inputs_x, endpoints = endpoint )

    # Check code.
    # Note that this is purely code based, not time based.
    if code.item() == 1: # Success
        poolscores[endpoint.uid] = moving_average_delta * poolscores[endpoint.uid] + (1 - moving_average_delta) * 1 # Converges towards 1
    else: # Failure
        poolscores[endpoint.uid] = moving_average_delta * poolscores[endpoint.uid] + (1 - moving_average_delta) * 0 # Converging towards 0
    
    # Return the responses
    return response

In [ ]:
# Create the axon RPC endpoint
axon = bittensor.axon( 
    wallet = poolwallet,
    forward_text = forward_text
)

# Start the axon RPC endpoint serving the forward function.
axon.start()

# Tell nobubaga that we exist and can start recieving messages.
axon.serve( subtensor = nobunaga )

In [ ]:

# Main loop.
sync_every_n_blocks = 10
while True:

    # Make an update every few blocks
    if poolchain.block % sync_every_n_blocks:

        # Set your weights onto the pool chain.
        poolchain.set_weights(
            wallet = poolwallet,
            uids = poolgraph.uids,
            weights = poolscores,
            wait_for_finalization = True 
        )

        # Get the latest nodes from your pool
        poolgraph.sync( subtensor = poolchain )
    else:
        time.sleep( 12 )
    

